### Install Requirements

In [1]:
! git clone -b stein_preprocessing https://github.com/MusondaSinkala/Machine-Learning-Project.git

fatal: destination path 'Machine-Learning-Project' already exists and is not an empty directory.


In [2]:
# ! pip install -r ./Machine-Learning-Project/requirements.txt

### Download Data from kaggle

In [3]:
# from google.colab import userdata
# import os

# os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
# os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [4]:
# %cd ../
# ! kaggle datasets download paultimothymooney/breast-histopathology-images

In [5]:
# ! unzip "/content/breast-histopathology-images.zip" -d "/content/Data"

### Extract features

In [6]:
%cd Machine-Learning-Project/

/content/Machine-Learning-Project


In [7]:
import cv2
import os
import pandas as pd
from preprocessing.feature_extraction import ImageFeatureExtractor, extract_details_from_path

In [8]:
df = pd.DataFrame(os.walk('/content/breast-histopathology-images'), columns=['directory_path', 'directory_name', 'files'])
df = df[df['files'].map(len) > 0]
df = df[['directory_path', 'files']]
df = df.explode('files')

In [9]:
feature_extraction = ImageFeatureExtractor()
def extract_features(directory: str, filename: str):
  img_path = os.path.join(directory, filename)
  img = cv2.imread(img_path)
  return feature_extraction(img) + list(extract_details_from_path(filename))



In [10]:
columns = ['MPI', 'SDPI', 'OTV', 'LM', 'UPP', 'LPP', 'PatientID', 'Xcoords', 'Ycoords', 'cancer']
df[columns] = df.apply(lambda x: extract_features(x['directory_path'], x['files']), axis=1, result_type='expand')

In [14]:
df.head()

,directory_path,files,MPI,SDPI,OTV,LM,UPP,LPP,PatientID,Xcoords,Ycoords,cancer
2,/content/breast-histopathology-images/16553/0,16553_idx5_x1901_y651_class0.png,216.6524,37.262769,191.0,1,77.24,2.56,16553,1901,651,0
2,/content/breast-histopathology-images/16553/0,16553_idx5_x1251_y201_class0.png,203.7960,48.483102,179.0,1,69.80,3.64,16553,1251,201,0
2,/content/breast-histopathology-images/16553/0,16553_idx5_x451_y1501_class0.png,183.1436,51.854093,182.0,1,46.84,9.24,16553,451,1501,0
2,/content/breast-histopathology-images/16553/0,16553_idx5_x1501_y351_class0.png,161.4748,39.490158,184.0,1,17.96,4.40,16553,1501,351,0
2,/content/breast-histopathology-images/16553/0,16553_idx5_x1401_y1601_class0.png,180.2956,24.432655,188.0,1,20.76,0.08,16553,1401,1601,0


In [15]:
df.to_csv('histopathological_data.csv', header=True, index=False)

In [16]:
df.shape

(555048, 12)

In [17]:
555048/2

277524.0